# Restore and serve EPICS PVs from the Archiver

Using a simple JSON file to seed a dict of `pvname:{'value':value}`, this will acquire values from the archiver, and run a simple EPICS server locally. 

In [1]:
from lcls_live.archiver import lcls_archiver_restore
import json
import os

In [2]:
# A list of some interesting PVs
PVDB_JSON = 'data/live_model_PVDB.json'

# Pick a proper time
ISOTIME = '2021-04-13T15:14:00.000000-07:00'

In [3]:
# Get starting PVDB
prefix = ''
pvdb = json.load(open(PVDB_JSON))

In [4]:
# Add some extras for fun
extra_pvs = ['BMLN:LI21:235:MOTR',
 'BMLN:LI24:805:MOTR',
 'ACCL:LI22:1:PDES',
 'ACCL:LI25:1:PDES']
for pv in extra_pvs:
    pvdb[pv] = {}

In [5]:
# Optional: update from live EPICS
# import epics
# 
# pvnames = list(pvdb)
# vals = epics.caget_many(list(pvdb))
# pvdb = {}
# for name, val in zip(pvnames, vals):
#     pvdb[name] = {'value':val}
# 

In [6]:
# Optional: 

# Open an SSH tunnel in a terminal like:
# ssh -D 8080 <SLAC username>@<SLAC machine>
# And then set:
os.environ['http_proxy']='socks5h://localhost:8080'
os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
os.environ['ALL_PROXY']='socks5h://localhost:8080'

In [7]:
# Update
new_pvdata = lcls_archiver_restore(list(pvdb), ISOTIME)
for k, v in new_pvdata.items():
#    print(pvdb[k]['value'], v)
    if v is None:
        print(k, 'Bad value:', v)
    else:
        pvdb[k]['value'] = v

Requesting: http://lcls-archapp.slac.stanford.edu/retrieval/data/getDataAtTime?at=2021-04-13T15:14:00.000000-07:00&includeProxies=true
KLYS:LI24:61:PHAS Bad value: None
KLYS:LI25:11:PHAS Bad value: None
KLYS:LI25:21:PHAS Bad value: None
KLYS:LI25:31:PHAS Bad value: None
KLYS:LI25:41:PHAS Bad value: None
KLYS:LI25:51:PHAS Bad value: None
KLYS:LI25:61:PHAS Bad value: None
KLYS:LI25:71:PHAS Bad value: None
KLYS:LI25:81:PHAS Bad value: None
KLYS:LI26:11:PHAS Bad value: None
KLYS:LI26:21:PHAS Bad value: None
KLYS:LI26:41:PHAS Bad value: None
KLYS:LI26:51:PHAS Bad value: None
KLYS:LI26:61:PHAS Bad value: None
KLYS:LI26:71:PHAS Bad value: None
KLYS:LI26:81:PHAS Bad value: None
KLYS:LI27:11:PHAS Bad value: None
KLYS:LI27:21:PHAS Bad value: None
KLYS:LI27:41:PHAS Bad value: None
KLYS:LI27:51:PHAS Bad value: None
KLYS:LI27:81:PHAS Bad value: None
KLYS:LI28:11:PHAS Bad value: None
KLYS:LI28:21:PHAS Bad value: None
KLYS:LI28:31:PHAS Bad value: None
KLYS:LI28:41:PHAS Bad value: None
KLYS:LI28:51:PH

In [8]:
# Dump to file for safekeeping
outfile = f'data/live_model_PVDB-{ISOTIME}.json'
json.dump(pvdb, open(outfile, 'w'))

# SimpleServer from pcaspy

This following is adapted from:
https://pcaspy.readthedocs.io/en/latest/tutorial.html

In [10]:
from pcaspy import Driver, SimpleServer

In [11]:
class myDriver(Driver):
    def __init__(self):
        super(myDriver, self).__init__()
        
    def getParam(self, reason):
        #print('myDriver.getParam', reason) 
        return super().getParam(reason)

In [12]:
server = SimpleServer()
server.createPV(prefix, pvdb)
driver = myDriver()

In [ ]:
while True:
    # process CA transactions
    server.process(0.1)

In [14]:
# cleanup
os.remove(outfile)